In [1]:
#need to import all of the code to be able to run powerflows and to build the grid.
import pandapower as pp
import pandas as pd
from math import sqrt
import matplotlib 
import pandapower.networks as nw
import pandapower.plotting as pplot
from math import acos,tan

In [2]:
#create all of the required information for the high, medium and low voltage components of the grid.
# High voltage 132 kV, medium voltage 11 kV, low voltage 0.415 kV
vh=132
vm=11
vl=0.415

#ADMD is prescribed in the Dominican Republic project remit and is 3kw and therefore = 0.003 watts
ADMD=0.003 

#Create the network
net = pp.create_empty_network() 

#Line values for medium voltage lines (11kV) 
#resistance (r), reactance (x) and capacitance (c) all are required for the lines
r_m=0.670
x_m=0.120
c_m=0.22

#Line values for low voltage lines (0.415kV)
#resistance (r), reactance (x) and capacitance (c) all are required for the lines
r_l=1.6
x_l=0.14
c_l=0.2

#The HV has been omitted from the scope of this project and any of the line information for the HV is not included. 

#Create the HV bus. This is the first part of the transformer
b1 = pp.create_bus(net,vn_kv=vh, name="Bus 1 HV", geodata=((0,50)))

pp.create_ext_grid(net, bus=b1, vm_pu=1, name="Swing bus")

#Create the next MV bus
b2 = pp.create_bus(net,vn_kv=vm, name="Bus 2 MV", geodata=((0,0)))

#create the line between the substation and the HV bus 
# pp.create_line_from_parameters(net,from_bus=b0, to_bus=b1,length_km=20, r_ohm_per_km = r_m, x_ohm_per_km = x_m, c_nf_per_km = 1e-9, max_i_ka=1000, name="b0 to b1")

#Trafo between HV and MV
#Info: https://pandapower.readthedocs.io/en/develop/std_types/basic.html#transformers (25 MVA 110/10 kV)
sn=0.5 #Rated appearent power
vkr=1 #short circuit voltage - real component
vk=12 #short circuit voltage
pfe=5 #iron losses
i0=0.07 #open loop losses

#Trafo between HV and MV
pp.create_transformer_from_parameters(net, hv_bus=b1, lv_bus=b2, sn_mva = sn, vn_hv_kv=vh, vn_lv_kv=vm, vkr_percent = vkr, vk_percent = vk, pfe_kw = pfe, i0_percent = i0, shift_degree=150, name ="Trafo 1")

#Create remaining MV buses
b3 = pp.create_bus(net,vn_kv=vm, name="Bus 3 MV", geodata=((-330,-300)))
b5 = pp.create_bus(net,vn_kv=vm, name="Bus 5 MV", geodata=((-110,-300)))
b7 = pp.create_bus(net,vn_kv=vm, name="Bus 6 MV", geodata=((110,-300)))
b9 = pp.create_bus(net,vn_kv=vm, name="Bus 7 MV", geodata=((330,-300)))

#Create the LV buses 
b4 = pp.create_bus(net,vn_kv=vl, name="Bus 4 LV", geodata=((-330,-350)))
b6 = pp.create_bus(net,vn_kv=vl, name="Bus 6 LV", geodata=((-110,-350)))
b8 = pp.create_bus(net,vn_kv=vl, name="Bus 8 LV", geodata=((110,-350)))
b10 = pp.create_bus(net,vn_kv=vl, name="Bus 10 LV", geodata=((330,-350)))

#Create all of the lines between the buses in the housing development 
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b3,length_km=0.445, r_ohm_per_km = r_m, x_ohm_per_km = x_m, c_nf_per_km = c_m, max_i_ka=0.178, name="b2 to b3")
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b5,length_km=0.319, r_ohm_per_km = r_m, x_ohm_per_km = x_m, c_nf_per_km = c_m, max_i_ka=0.178,  name="b2 to b5")
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b7,length_km=0.319, r_ohm_per_km = r_m, x_ohm_per_km = x_m, c_nf_per_km = c_m, max_i_ka=0.178,  name="b2 to b7")
pp.create_line_from_parameters(net,from_bus=b2, to_bus=b9,length_km=0.445, r_ohm_per_km = r_m, x_ohm_per_km = x_m, c_nf_per_km = c_m, max_i_ka=0.178, name="b2 to b9")


#Trafo between MV and LV 
#Info: https://pandapower.readthedocs.io/en/develop/std_types/basic.html#transformers (0.4 MVA 10/0.4 kV)
sn=0.4 #Rated appearent power
vkr=1.325 #short circuit voltage - real component
vk=4 #short circuit voltage
pfe=0.95 #iron losses
i0=0.3375 #open loop losses

#Create the Trafo between all of the MV and LV buses
pp.create_transformer_from_parameters(net, shift_degree=150, hv_bus=b3, lv_bus=b4, sn_mva = sn, vn_hv_kv=vm, vn_lv_kv=vl, vkr_percent = vkr, vk_percent = vk, pfe_kw = pfe, i0_percent = i0, name ="Trafo 2")
pp.create_transformer_from_parameters(net, shift_degree=150, hv_bus=b5, lv_bus=b6, sn_mva = sn, vn_hv_kv=vm, vn_lv_kv=vl, vkr_percent = vkr, vk_percent = vk, pfe_kw = pfe, i0_percent = i0, name ="Trafo 3")
pp.create_transformer_from_parameters(net, shift_degree=150, hv_bus=b7, lv_bus=b8, sn_mva = sn, vn_hv_kv=vm, vn_lv_kv=vl, vkr_percent = vkr, vk_percent = vk, pfe_kw = pfe, i0_percent = i0, name ="Trafo 4")
pp.create_transformer_from_parameters(net, shift_degree=150, hv_bus=b9, lv_bus=b10, sn_mva = sn, vn_hv_kv=vm, vn_lv_kv=vl, vkr_percent = vkr, vk_percent = vk, pfe_kw = pfe, i0_percent = i0, name ="Trafo 5")

4

In [3]:
pp.runpp(net)

In [4]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.008788,-0.00383
1,0.998960,-0.081876,0.000000,0.00000
2,0.998957,-0.081765,0.000000,0.00000
3,0.998958,-0.081797,0.000000,0.00000
4,0.998958,-0.081797,0.000000,0.00000
5,0.998957,-0.081765,0.000000,0.00000
6,0.998896,-0.083423,0.000000,0.00000
7,0.998897,-0.083454,0.000000,0.00000
8,0.998897,-0.083454,0.000000,0.00000
9,0.998896,-0.083423,0.000000,0.00000


In [5]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.000948,0.000953,-0.000948,-0.000957,4.472077e-09,-0.000004,0.000071,0.000071,0.000071,0.99896,-0.081876,0.998957,-0.081765,0.039763
1,0.000948,0.000954,-0.000948,-0.000957,3.207600e-09,-0.000003,0.000071,0.000071,0.000071,0.99896,-0.081876,0.998958,-0.081797,0.039763
2,0.000948,0.000954,-0.000948,-0.000957,3.207600e-09,-0.000003,0.000071,0.000071,0.000071,0.99896,-0.081876,0.998958,-0.081797,0.039763
3,0.000948,0.000953,-0.000948,-0.000957,4.472077e-09,-0.000004,0.000071,0.000071,0.000071,0.99896,-0.081876,0.998957,-0.081765,0.039763


In [6]:
#Create the LV buses 
hb0 = pp.create_bus(net,vn_kv=vl, name="housing bus 0", geodata=((-390,-75)))
hb1 = pp.create_bus(net,vn_kv=vl, name="housing bus 1", geodata=((-390,-225)))
hb2 = pp.create_bus(net,vn_kv=vl, name="housing bus 2", geodata=((-390,-375)))
hb3 = pp.create_bus(net,vn_kv=vl, name="housing bus 3", geodata=((-390,-525)))
hb4 = pp.create_bus(net,vn_kv=vl, name="housing bus 4", geodata=((-270,-75)))
hb5 = pp.create_bus(net,vn_kv=vl, name="housing bus 5", geodata=((-270,-225)))
hb6 = pp.create_bus(net,vn_kv=vl, name="housing bus 6", geodata=((-270,-375)))

In [7]:
#Create the lines between all of the domestic buses on the left branch 
pp.create_line_from_parameters(net,from_bus=b4, to_bus=hb1,length_km=0.075, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096, name="b4,hb1")
pp.create_line_from_parameters(net,from_bus=hb1, to_bus=hb0,length_km=0.225, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096,  name="hb1,hb0")
pp.create_line_from_parameters(net,from_bus=b4, to_bus=hb2,length_km=0.075, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096,  name="b4,hb2")
pp.create_line_from_parameters(net,from_bus=hb2, to_bus=hb3,length_km=0.0225, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096, name="hb2,hb3")

#create all the lines on the right hand side
pp.create_line_from_parameters(net,from_bus=b4, to_bus=hb5,length_km=0.075, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096,  name="hb1,hb0")
pp.create_line_from_parameters(net,from_bus=hb5, to_bus=hb4,length_km=0.225, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096,  name="b4,hb2")
pp.create_line_from_parameters(net,from_bus=b4, to_bus=hb6,length_km=0.075, r_ohm_per_km = r_l, x_ohm_per_km = x_l, c_nf_per_km = c_l, max_i_ka=0.096, name="hb2,hb3")

10

In [8]:
#Create all of the loads 

In [9]:
#43 houses per trafo
admd=0.003
safety_factor=1.5 #give it a higher rate load
sn_mva =43*admd*1.5 #rated apparent power

six_house_plot = [hb0, hb1, hb2, hb3, hb4, hb5]

loads=[]

power_factor=0.9

p_mw_per_house=0.003
q_mvar_per_house=p_mw_per_house*tan(acos(power_factor))
p_mw_PV_per_house=-0.0012
q_mvar_PV_per_house=p_mw_PV_per_house*tan(acos(power_factor))
number_of_houses_normal=6
number_of_houses_extra=7

#Create the loads for all of the houses in a development branch (not including the 7 house bus)
for i in range(0,6):
    loads.append(pp.create_load(net,bus=six_house_plot[i], p_mw=p_mw_per_house*number_of_houses_normal, q_mvar=q_mvar_per_house*number_of_houses_normal,  name=f'Combined load at housing bus{i}'))
    #loads.append(pp.create_load(net,bus=six_house_plot[i], p_mw=p_mw_PV_per_house*number_of_houses_normal, q_mvar=q_mvar_PV_per_house*number_of_houses_normal,  name=f'Combined PV at housing bus{i}'))

#For the 7 house bus
loads.append(pp.create_load(net,bus=hb6, p_mw=p_mw_per_house*number_of_houses_extra, q_mvar=q_mvar_per_house*number_of_houses_extra,  name='Combined load at seven house bus'))
#loads.append(pp.create_load(net,bus=hb6, p_mw=p_mw_PV_per_house*number_of_houses_extra, q_mvar=q_mvar_PV_per_house*number_of_houses_extra,  name='Combined PV load at seven house bus'))

In [10]:
pp.runpp(net)

In [11]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.137269,0.066230,-0.137209,-0.066223,5.963417e-05,0.000007,0.008165,0.008165,0.008165,0.979706,-1.911855,0.979330,-1.905727,4.587243
1,0.000912,0.000918,-0.000912,-0.000921,3.085098e-09,-0.000003,0.000069,0.000069,0.000069,0.979706,-1.911855,0.979704,-1.911776,0.038997
2,0.000912,0.000918,-0.000912,-0.000921,3.085098e-09,-0.000003,0.000069,0.000069,0.000069,0.979706,-1.911855,0.979704,-1.911776,0.038997
3,0.000912,0.000917,-0.000912,-0.000921,4.301343e-09,-0.000004,0.000069,0.000069,0.000069,0.979706,-1.911855,0.979703,-1.911744,0.038997
4,0.038367,0.017643,-0.037042,-0.017527,1.324842e-03,0.000116,0.060664,0.060664,0.060664,0.968431,-2.553686,0.939772,-1.927038,63.191629
5,0.019042,0.008809,-0.018000,-0.008718,1.041838e-03,0.000091,0.031059,0.031059,0.031059,0.939772,-1.927038,0.895845,-0.910873,32.353188
6,0.037361,0.017555,-0.036095,-0.017444,1.265982e-03,0.000111,0.059301,0.059301,0.059301,0.968431,-2.553686,0.940501,-1.927524,61.771939
7,0.018095,0.008726,-0.018000,-0.008718,9.537313e-05,0.000008,0.029717,0.029717,0.029717,0.940501,-1.927524,0.936311,-1.830379,30.954935
8,0.038367,0.017643,-0.037042,-0.017527,1.324842e-03,0.000116,0.060664,0.060664,0.060664,0.968431,-2.553686,0.939772,-1.927038,63.191629
9,0.019042,0.008809,-0.018000,-0.008718,1.041838e-03,0.000091,0.031059,0.031059,0.031059,0.939772,-1.927038,0.895845,-0.910873,32.353188


In [12]:
net.res_load

,p_mw,q_mvar
0,0.018,0.008718
1,0.018,0.008718
2,0.018,0.008718
3,0.018,0.008718
4,0.018,0.008718
5,0.018,0.008718
6,0.021,0.010171


In [13]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.145423,-0.075223
1,0.979706,-1.911855,0.000000,0.000000
2,0.979330,-1.905727,0.000000,0.000000
3,0.979704,-1.911776,0.000000,0.000000
4,0.979704,-1.911776,0.000000,0.000000
5,0.979703,-1.911744,0.000000,0.000000
6,0.968431,-2.553686,0.000000,0.000000
7,0.979644,-1.913433,0.000000,0.000000
8,0.979644,-1.913433,0.000000,0.000000
9,0.979643,-1.913402,0.000000,0.000000


In [14]:
#Now need to add all of the dummy loads for the remaining branches in the development
#As they come out of an additional 3 buses, the compounded loads can be seen below as Bus 6, 8 and 10. 

#Create the loads for the remaining development branches (from bus 6)
loads.append(pp.create_load(net,bus=b6, p_mw=p_mw_per_house*43, q_mvar=q_mvar_per_house*43,  name='Combined dummy load at bus 6'))
loads.append(pp.create_load(net,bus=b6, p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name='Combined dummy PV at bus 6'))

#Create dummy load from bus 8
loads.append(pp.create_load(net,bus=b8, p_mw=p_mw_per_house*43, q_mvar=q_mvar_per_house*43,  name='Combined dummy load at bus 8'))
loads.append(pp.create_load(net,bus=b8, p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name='Combined dummy PV at bus 8'))

#Create dummy load from bus 10
loads.append(pp.create_load(net,bus=b10, p_mw=p_mw_per_house*43, q_mvar=q_mvar_per_house*43,  name='Combined dummy load at bus 10'))
loads.append(pp.create_load(net,bus=b10, p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name='Combined dummy PV at bus 10'))

In [15]:
net.bus

,name,vn_kv,type,zone,in_service
0,Bus 1 HV,132.000,b,None,True
1,Bus 2 MV,11.000,b,None,True
2,Bus 3 MV,11.000,b,None,True
3,Bus 5 MV,11.000,b,None,True
4,Bus 6 MV,11.000,b,None,True
5,Bus 7 MV,11.000,b,None,True
6,Bus 4 LV,0.415,b,None,True
7,Bus 6 LV,0.415,b,None,True
8,Bus 8 LV,0.415,b,None,True
9,Bus 10 LV,0.415,b,None,True


In [16]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Combined load at housing bus0,10,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
1,Combined load at housing bus1,11,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
2,Combined load at housing bus2,12,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
3,Combined load at housing bus3,13,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
4,Combined load at housing bus4,14,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
5,Combined load at housing bus5,15,0.0180,0.008718,0.0,0.0,NaN,1.0,True,wye
6,Combined load at seven house bus,16,0.0210,0.010171,0.0,0.0,NaN,1.0,True,wye
7,Combined dummy load at bus 6,7,0.1290,0.062478,0.0,0.0,NaN,1.0,True,wye
8,Combined dummy PV at bus 6,7,-0.0516,-0.024991,0.0,0.0,NaN,1.0,True,wye
9,Combined dummy load at bus 8,8,0.1290,0.062478,0.0,0.0,NaN,1.0,True,wye
